## Ambiente Inicial

In [84]:
# librerias
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime


In [85]:
script = os.getcwd()
file = os.path.join(script, "data", "BD_IncendiosSNIF_2015-2023.csv")
df = pd.read_csv(file, encoding="latin-1")

C:\Users\Franc\AppData\Local\Temp\ipykernel_13212\4104928706.py:3: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, encoding="latin-1")


In [86]:
df.sample(3)

,Año,Latitud,Longitud,Estado,Municipio,Región,Predio,Fecha Inicio,Fecha Termino,Duración días,...,Renuevo,Arbustivo,Herbáceo,Hojarasca,Total hectáreas,Tamaño,Detección,Llegada,Duración,Causa
5587,2016,19.342139,-102.107528,Michoacán,Uruapan,Occidente,Ej. Matanguaran (Cerro de la Hacienda),11/03/2016,11/03/2016,1 Día,...,0.0,1.0,2.0,0.0,3.00,0 a 5 Hectáreas,0.020833333,0.041666667,0.291666667,Actividades Agrícolas
37141,2020,19.468611,-98.726278,México,Texcoco,Centro,Ejido santa maria tecuanulco/La peña,14/04/2020,14/04/2020,1 Día,...,0,1.0,0.0,0.0,1.00,0 a 5 Hectáreas,0.041666667,0.041666667,3.0625,Actividades ilícitas
53516,2022,22.177689,-103.877319,Jalisco,Mezquitic,Occidente,El Sombrero,18/05/2022,24/05/2022,4 a 7 Días,...,16.11,0.0,0.0,100.0,166.11,Mayor a 100 Hectáreas,00:10,00:35,29:05:00,Desconocidas


## Exploración de datos y limpieza

Comenzaremos por eliminar aquellos espacios en blanco y cambiarlos por "0" para tratarlos después.

In [87]:
df.replace(" ", 0, inplace=True)
df["Renuevo"] = df["Renuevo"].astype(float) 

Ahora corrobaremos que nuestros datos hayan sido leídos correctamente y tengamos los tipos de datos correctos

In [88]:
# Observamos las descripciones estadísticas de nuestras variables númericas
df.describe()

,Año,Latitud,Longitud,Arbolado Adulto,Renuevo,Arbustivo,Herbáceo,Hojarasca,Total hectáreas
count,62441.000000,62441.000000,62441.000000,54758.000000,61506.000000,61918.000000,62002.00000,62141.000000,62441.00000
mean,2019.131324,20.597185,-100.816937,2.204616,1.738875,19.018733,30.34795,23.741466,80.52891
std,2.494713,3.206582,4.395817,40.889945,19.549905,187.361226,267.26755,167.638854,515.37220
min,2015.000000,14.696064,-118.286697,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000
25%,2017.000000,19.119222,-103.558372,0.000000,0.000000,0.000000,0.00000,0.000000,1.50000
50%,2019.000000,19.552250,-99.927250,0.000000,0.000000,0.500000,1.00000,0.000000,5.00000
75%,2021.000000,20.892500,-98.712803,0.000000,0.000000,4.000000,6.00000,3.590000,23.65000
max,2023.000000,32.599875,-86.798778,4893.000000,1271.710000,17382.420000,17220.15000,8717.330000,23809.00000


In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62441 entries, 0 to 62440
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Año               62441 non-null  int64  
 1   Latitud           62441 non-null  float64
 2   Longitud          62441 non-null  float64
 3   Estado            62441 non-null  object 
 4   Municipio         62441 non-null  object 
 5   Región            62441 non-null  object 
 6   Predio            62441 non-null  object 
 7   Fecha Inicio      62438 non-null  object 
 8   Fecha Termino     62441 non-null  object 
 9   Duración días     62440 non-null  object 
 10  Tipo de incendio  62441 non-null  object 
 11  Tipo Vegetación   62441 non-null  object 
 12  Régimen de fuego  62441 non-null  object 
 13  Tipo impacto      62441 non-null  object 
 14  Arbolado Adulto   54758 non-null  float64
 15  Renuevo           61506 non-null  float64
 16  Arbustivo         61918 non-null  float6

En esta sección trataremos los valores categoricos para normalizar nuestras respuestas, ya que hay mucha variabilidad entre respuestas.

In [90]:
# Normalización de Columnas con valores categoricos variables
estados = {
    'Ciudad De México':'Ciudad de México',
    'México':'Ciudad de México'
}

df["Estado"] = df["Estado"].replace(estados) 

In [91]:
municipios = { 
    "Alamos":"Álamos",
    "Angel Albino Corzo":"Ángel Albino Corzo"
}
df['Municipio'] = df['Municipio'].replace(municipios)

In [92]:
df["Duración días"] = df["Duración días"].replace("#¡REF!", '0') 

In [93]:
incendios = {
    'superficial':'Superficial',
    'SUPERFICIAL':'Superficial',
    'mixto':'Mixto',
    '0':"Desconocido"
}
df["Tipo de incendio"] = df["Tipo de incendio"].replace(incendios)

In [94]:
vegetacion = {
    '0':"Desconocida",
    "Bosque Mesofilo - BM": "Bosque Mesófilo - BM",
    "Matorral Desertico Micrófilo":"Matorral Desértico Micrófilo",
    "Matorral Desertico Rosetófilo":"Matorral Desértico Rosetófilo",
    "Matorral Sarco-Crasicaule":"Matorral Sarcocrasicaule",
    "Pastizal Gypsófilo":"Pastizal Gipsófilo",
    "Vegetación Halofila Hidrófila":"Vegetación Halófila Hidrófila",
}
df['Tipo Vegetación'] = df['Tipo Vegetación'].replace(vegetacion)

In [95]:
impactos = {'Impacto Mínimo':"Impacto Minimo",
'impacto mínimo':"Impacto Minimo",
'impacto Mínimo':"Impacto Minimo",
'Impacto mínimo':"Impacto Minimo",
'Impacto MÍnImo':"Impacto Minimo",
'impacto moderado':'Impacto Moderado',
'Impacto moderado':'Impacto Moderado',
'impacto severo':'Impacto Severo',
'Impacto severo':'Impacto Severo',
'0':"Impacto Nulo",
}
df["Tipo impacto"] = df["Tipo impacto"].replace(impactos)

In [96]:
df["Tamaño"] = df["Tamaño"].replace({'0':"Desconocido"})

In [97]:
causas = {
    'fumadores':"Fumadores",
    'actividades pecuarias':"Actividades Pecuarias",
    'Actividades pecuarias':"Actividades Pecuarias",
    'cazadores':"Cazadores",
    'actividades ilícitas':"Actividades Ilícitas",
    'Quema de basureros':"Quema de Basureros",
    'Otras actividades productivas':"Otras Actividades Productivas",
    'Residuos de aprovechamiento forestal':"Residuos de Aprovechamiento Forestal",
    'fogatas':"Fogatas",
    'Actividades agrícolas':"Actividades Agrícolas",
    'DESCONOCIDAS':"Desconocidas",
    'Actividades ilícitas': "Actividades Ilícitas"
}
df["Causa"] = df["Causa"].replace(causas) 

Una vez que ya estén normalizadas las respuestas procederemos a limpiar los datos nulos o con valores "0" que asignamos anteriormente.

In [98]:
# Mostramos si existe aunque sea 1 dato nulo en cada columna
df.isnull().any()

Año                 False
Latitud             False
Longitud            False
Estado              False
Municipio           False
Región              False
Predio              False
Fecha Inicio         True
Fecha Termino       False
Duración días        True
Tipo de incendio    False
Tipo Vegetación     False
Régimen de fuego    False
Tipo impacto        False
Arbolado Adulto      True
Renuevo              True
Arbustivo            True
Herbáceo             True
Hojarasca            True
Total hectáreas     False
Tamaño              False
Detección           False
Llegada             False
Duración            False
Causa               False
dtype: bool

Como podemos observar en los resultados obtenidos arriba, tenemos algunas columnas que cuentan con datos nulos, las cuales son: *Fecha Inicio, Duración Días, Arbolado Adulto, Renuevo, Arbustivo, Herbáceo, Hojarasca*. <br>

Para ajustar los valores nulos en las columnas de fechas simplemente se observarán los datos en los registros de "Fecha Termino" y "Duración días", para inferir las fechas faltantes.

In [99]:
# Corrección de valores faltantes en Fechas
df.loc[62438, "Fecha Inicio"] = "27/02/2015"
df.loc[62439, "Fecha Inicio"] = "13/03/2015"
df.loc[62440, "Fecha Inicio"] = "16/03/2015"
# Correción duración de días
df.loc[2064, "Duración días"] = '1 Día'
# Correción Detección
df.loc[35812, "Detección"] = '00:05:00'


Ahora, las siguientes columnas tiene algo de similar en cuanto a la razón de sus valores nulos, y es que no existe información para rellenar el espacio correspondiente, por lo cual al ser información faltante simplemente se le asignará un 0.

In [100]:
# Corrección de valores faltantes en Arbolados Adultos
df["Arbolado Adulto"] = df["Arbolado Adulto"].fillna(0)
# Corrección de valores faltantes en Renuevo
df["Renuevo"] = df["Renuevo"].fillna(0)
# Corrección de valores faltantes en Arbustivos
df["Arbustivo"] = df["Arbustivo"].fillna(0)
# Corrección de valores en Herbáceos
df["Herbáceo"] = df["Herbáceo"].fillna(0)
# Correción de valores en Hojarascas
df["Hojarasca"] = df["Hojarasca"].fillna(0)

In [101]:
# Correción de valores con errores de digitación
df["Detección"] = df["Detección"].str.replace(";:", ":")
df["Detección"] = df["Detección"].str.replace("::", ":")
df["Llegada"] = df["Llegada"].str.replace(";", ":")
df["Duración"] = df["Duración"].str.replace(";", ":")
df["Llegada"] = df["Llegada"].str.replace(r':000\b', ":00", regex=True)
df["Llegada"] = df["Llegada"].str.replace("::", ":")
df["Duración"] = df["Duración"].str.replace(" a.m.", "")
df["Duración"] = df["Duración"].str.replace(" a. m.", "")
df["Duración"] = df["Duración"].str.replace("  ", "")
df["Duración"] = df["Duración"].str.replace(" ", "")
#26:28:00 a.m.  a. m.

In [102]:
df[["Fecha Inicio", "Fecha Termino"]] = df[["Fecha Inicio", "Fecha Termino"]].apply(pd.to_datetime, format="%d/%m/%Y")

In [103]:
df.sample(3)

,Año,Latitud,Longitud,Estado,Municipio,Región,Predio,Fecha Inicio,Fecha Termino,Duración días,...,Renuevo,Arbustivo,Herbáceo,Hojarasca,Total hectáreas,Tamaño,Detección,Llegada,Duración,Causa
36390,2020,19.073806,-100.01675,Ciudad de México,Temascaltepec,Centro,Carboneras /Río Verde,2020-04-01,2020-04-01,1 Día,...,0.0,4.0,5.0,0.0,9.0,6 a 10 Hectáreas,0.010416667,0.024305556,0.170138889,Cazadores
3733,2016,19.235583,-99.43875,Ciudad de México,Tianguistenco,Centro,Comunal Tilapa,2016-01-23,2016-01-23,1 Día,...,0.0,2.0,1.0,0.0,3.0,0 a 5 Hectáreas,0.0625,0.229166667,0.375,Actividades Pecuarias
2734,2015,26.208333,-106.79725,Chihuahua,Guadalupe y Calvo,Norte,Ej. Chinatu,2015-05-12,2015-05-12,1 Día,...,0.0,0.0,2.0,1.0,3.0,0 a 5 Hectáreas,0.003472222,0.013888889,0.061111111,Actividades Agrícolas


Ahora comprobaremos que los datos hayan sido tratados correctamente:

In [104]:
df[df["Llegada"].str.contains(r':000\b')]

,Año,Latitud,Longitud,Estado,Municipio,Región,Predio,Fecha Inicio,Fecha Termino,Duración días,...,Renuevo,Arbustivo,Herbáceo,Hojarasca,Total hectáreas,Tamaño,Detección,Llegada,Duración,Causa


In [105]:
df[df["Detección"].str.contains(r'0\.\d+$')].sample(3)

#df[df["Detección"].str.contains(r'\d{1}\.\d+')].sample(15)

,Año,Latitud,Longitud,Estado,Municipio,Región,Predio,Fecha Inicio,Fecha Termino,Duración días,...,Renuevo,Arbustivo,Herbáceo,Hojarasca,Total hectáreas,Tamaño,Detección,Llegada,Duración,Causa
34056,2019,23.628544,-99.896917,Tamaulipas,Miquihuana,Noreste,Ejido Servando Canales,2019-06-02,2019-06-05,4 a 7 Días,...,0.0,39.0,0.0,0.0,39.0,21 a 50 Hectáreas,0.006944444,0.111111111,0.208333333,Naturales
27020,2018,25.890222,-106.863778,Chihuahua,Guadalupe y Calvo,Norte,Ej. Redondeados,2018-06-05,2018-06-05,1 Día,...,0.0,2.0,3.0,7.0,12.0,11 a 20 Hectáreas,0.041666667,0.604166667,0.875,Actividades Agrícolas
1194,2015,16.609806,-96.743944,Oaxaca,San Miguel Ejutla,Centro,San Miguel Ejutla,2015-03-08,2015-03-09,2 a 3 Días,...,0.0,2.0,6.0,2.0,10.0,6 a 10 Hectáreas,0.003472222,0.059027778,0.118055556,Fumadores


In [106]:
df[df["Detección"].str.contains(r'\d{1}\.\d+')].sample(5)

,Año,Latitud,Longitud,Estado,Municipio,Región,Predio,Fecha Inicio,Fecha Termino,Duración días,...,Renuevo,Arbustivo,Herbáceo,Hojarasca,Total hectáreas,Tamaño,Detección,Llegada,Duración,Causa
16273,2017,19.689639,-101.542889,Michoacán,Quiroga,Occidente,C.I De Santa Fe De La Laguna,2017-04-12,2017-04-13,2 a 3 Días,...,0.0,4.0,7.40,2.0,16.40,11 a 20 Hectáreas,0.003472222,0.024305556,0.065972222,Actividades Agrícolas
6513,2016,19.612528,-99.370417,Ciudad de México,Nicolás Romero,Centro,Comunal Cahuacan,2016-03-29,2016-03-29,1 Día,...,0.0,8.0,0.00,0.0,8.00,6 a 10 Hectáreas,0.013888889,0.03125,0.208333333,Actividades Ilícitas
3452,2015,31.992381,-116.696206,Baja California,Ensenada,Noroeste,KM 17 Carretera Guadalupe - El Porvenir,2015-08-28,2015-08-28,1 Día,...,0.0,3.0,0.00,0.0,3.00,0 a 5 Hectáreas,0.104166667,0.027777778,0.302083333,Quema de Basureros
17758,2017,32.452194,-116.790278,Baja California,Tijuana,Noroeste,Autodromo,2017-05-01,2017-05-02,2 a 3 Días,...,0.0,57.0,280.00,0.0,337.00,Mayor a 100 Hectáreas,0.003472222,0.053472222,0.14375,Actividades Ilícitas
31096,2019,16.720556,-92.706667,Chiapas,Zinacantán,Sureste,Particular,2019-04-19,2019-04-20,2 a 3 Días,...,0.0,0.0,0.41,1.0,1.91,0 a 5 Hectáreas,0.006944444,0.013888889,0.083333333,Actividades Agrícolas


In [107]:
df[df["Detección"].str.contains(r'\d{2}\:\d{2}')].sample(3)

,Año,Latitud,Longitud,Estado,Municipio,Región,Predio,Fecha Inicio,Fecha Termino,Duración días,...,Renuevo,Arbustivo,Herbáceo,Hojarasca,Total hectáreas,Tamaño,Detección,Llegada,Duración,Causa
51112,2022,19.856203,-103.000653,Jalisco,Mazamitla,Occidente,El Corral de la Mula,2022-04-06,2022-04-08,2 a 3 Días,...,0.0,1.32,0.00,0.00,31.32,21 a 50 Hectáreas,00:05,00:45,03:30,Desconocidas
51557,2022,21.329183,-101.295817,Guanajuato,San Felipe,Occidente,San José de Los Barcos,2022-04-15,2022-04-15,1 Día,...,0.0,0.00,17.54,70.15,87.69,51 a 100 Hectáreas,00:00,00:13,02:00,Desconocidas
55813,2023,18.941842,-98.448486,Puebla,Tianguismanalco,Centro,Ejido San Juan Tianguismanalco / Cerro El Corona,2023-02-25,2023-02-25,1 Día,...,0.0,0.00,5.00,0.00,5.00,0 a 5 Hectáreas,00:20:00,00:50:00,93:00:00,Intencional


In [108]:
def convert_to_hours(time_str):
    """Función para transformar diferentes tipos de registros en un formato de hora
    HH:mm:ss.
    """
    if re.match(r'^\d{3,}:\d{2}:\d{2}$', time_str):  # Sí el del tipo 608:30:00
        return "eliminar"
    elif re.match(r'^\d{2}:\d{2}:\d{2}$', time_str):  # Si es del tipo 00:30:00 
        horas, minutos, segundos = map(int, time_str.split(':'))
        if horas < 24:
            return time_str
        else:
            return "eliminar"
    elif re.match(r'^\d{1}\:\d{2}\:\d{2}$', time_str):# Sí es del tipo 13:25:43
        return time_str.zfill(8)
    elif re.match(r'^\d{2}:\d{2}$', time_str):# Si es del tipo 02:25
        return time_str + ":00"
    elif re.match(r'\d+\.\d+$', time_str): # 3.165
        horas = float(time_str) * 24
        # Calcula las horas enteras
        horas_enteras = int(horas)
        # Calcula los minutos y segundos
        minutos = int((horas - horas_enteras) * 60)
        segundos = int(((horas - horas_enteras) * 60 - minutos) * 60)
        # Formatea la salida como HH:MM:SS
        if horas_enteras <24:
            return "{:02d}:{:02d}:{:02d}".format(horas_enteras, minutos, segundos)
        else:
            return "eliminar"
    elif re.match(r"\d+$", time_str): # Sí es del tipo 1,25,79
        if int(time_str) <24:
            return "00:00:00"
        else:
            return "eliminar"
    elif re.match(r"^\d{3,}\:\d{2}$", time_str):
        return "eliminar"
    else:
        raise ValueError(f"Formato no reconocido: {time_str}")

In [109]:
columnas = ["Detección", "Llegada", "Duración"]

df["Detección"] = df["Detección"].apply(convert_to_hours)
df["Llegada"] = df["Llegada"].apply(convert_to_hours)
#df["Duración"] = df["Duración"].apply(convert_to_hours)

In [110]:
df["Detección"].unique()

array(['00:09:59', '00:08:00', '00:04:59', '00:06:00', '00:20:00',
       '00:29:59', '00:13:00', '00:15:00', '00:02:00', '01:20:00',
       '00:06:59', '00:17:00', '01:14:59', '00:24:00', '00:33:00',
       '00:09:00', '00:44:00', '04:00:00', '00:11:59', '00:35:00',
       '00:02:59', '00:27:59', '01:25:00', '00:18:59', '14:19:59',
       '13:59:59', '04:50:00', '00:26:59', '13:00:00', '01:00:00',
       '16:30:00', '01:59:59', '00:24:59', '02:30:00', '00:45:00',
       '03:29:59', '01:30:00', '04:59:59', '03:00:00', 'eliminar',
       '00:40:00', '04:45:00', '19:59:59', '18:00:00', '00:00:59',
       '00:49:59', '00:00:19', '01:39:59', '00:13:59', '01:09:59',
       '22:59:59', '08:10:00', '00:53:00', '01:54:59', '02:44:59',
       '02:10:00', '06:24:00', '00:36:00', '00:18:00', '00:00:00',
       '10:00:00', '00:31:00', '14:05:00', '13:38:59', '17:10:00',
       '06:29:59', '00:04:00', '16:59:59', '00:04:40', '00:20:12',
       '00:00:40', '00:00:25', '00:05:05', '00:09:50', '06:00:

In [111]:
df[~df["Llegada"].str.contains(r'^\d{3}\:\d{2}\:\d{2}$')]

,Año,Latitud,Longitud,Estado,Municipio,Región,Predio,Fecha Inicio,Fecha Termino,Duración días,...,Renuevo,Arbustivo,Herbáceo,Hojarasca,Total hectáreas,Tamaño,Detección,Llegada,Duración,Causa
0,2015,20.369306,-104.546556,Jalisco,Atenguillo,Occidente,El cuatillo,2015-01-01,2015-01-02,2 a 3 Días,...,0.0,0.00,0.00,1.5,1.50,0 a 5 Hectáreas,00:09:59,00:35:00,0.291666667,Desconocidas
1,2015,19.307297,-99.092594,Ciudad de México,Xochimilco,Centro,Ejidos de Xochimilco,2015-01-01,2015-01-01,1 Día,...,0.0,1.50,0.00,0.0,1.50,0 a 5 Hectáreas,00:08:00,00:40:00,0.118055556,Actividades Ilícitas
2,2015,19.273358,-99.056281,Ciudad de México,Xochimilco,Centro,San Gregorio Atlapulco,2015-01-01,2015-01-01,1 Día,...,0.0,1.00,0.00,0.0,1.00,0 a 5 Hectáreas,00:04:59,00:15:00,1.013888889,Actividades Ilícitas
3,2015,19.275750,-98.114833,Tlaxcala,Chiautempan,Centro,Tlalcuapan,2015-01-01,2015-01-01,1 Día,...,0.0,0.00,0.50,0.0,0.50,0 a 5 Hectáreas,00:06:00,00:24:59,0.070138889,Actividades Pecuarias
4,2015,19.540444,-97.231556,Veracruz,Perote,Centro,Ejido Perote,2015-01-01,2015-01-01,1 Día,...,0.0,0.25,0.00,0.0,0.25,0 a 5 Hectáreas,00:20:00,00:35:00,1.083333333,Fumadores
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62436,2023,19.047511,-97.309186,Puebla,Tlachichuca,Centro,A.N.P. / El Plan del Venado,2023-12-30,2023-12-30,1 Día,...,0.0,0.00,0.00,3.0,3.00,0 a 5 Hectáreas,00:30:00,05:00:00,19:00:00,Actividades Pecuarias
62437,2023,19.331944,-98.990222,Ciudad de México,Iztapalapa,Centro,San Miguel Teotongo,2023-12-31,2023-12-31,1 Día,...,0.0,0.00,2.89,0.0,2.89,0 a 5 Hectáreas,00:10:00,02:50:00,11:30:00,Festividades y rituales
62438,2015,19.576389,-99.174167,Ciudad de México,Tlalnepantla de Baz,Centro,Particular Jardinez del Recuerdo,2015-02-27,2015-02-27,0,...,0.0,0.32,0.48,0.0,0.80,0 a 5 Hectáreas,00:30:00,00:00:00,06:00:00,Actividades Ilícitas
62439,2015,19.623222,-97.188250,Veracruz,Villa Aldama,Centro,Ejido Villa Aldama,2015-03-13,2015-03-13,0,...,0.0,0.50,0.00,0.0,0.50,0 a 5 Hectáreas,00:03:00,00:00:00,04:00:00,Fumadores


In [112]:
df.sample(20)

,Año,Latitud,Longitud,Estado,Municipio,Región,Predio,Fecha Inicio,Fecha Termino,Duración días,...,Renuevo,Arbustivo,Herbáceo,Hojarasca,Total hectáreas,Tamaño,Detección,Llegada,Duración,Causa
6381,2016,19.829111,-97.824583,Puebla,Tetela de Ocampo,Centro,Soyotitla,2016-03-27,2016-03-27,1 Día,...,0.00,1.80,10.00,0.00,11.80,11 a 20 Hectáreas,00:45:00,00:45:59,0.28125,Actividades Ilícitas
45989,2021,23.136108,-98.353889,Tamaulipas,Casas,Noreste,Ejido Eduardo Benavides,2021-04-24,2021-05-03,Mayor de 7 Días,...,0.00,100.00,0.00,124.00,234.00,Mayor a 100 Hectáreas,00:09:59,00:20:00,0.201388889,Actividades Agrícolas
49080,2022,20.782481,-103.642217,Jalisco,Zapopan,Occidente,Los Bailarines,2022-03-07,2022-03-07,1 Día,...,0.00,0.00,0.00,1.00,2.07,0 a 5 Hectáreas,00:05:00,00:16:00,00:34,Actividades Ilícitas
9307,2016,21.806528,-100.747500,San Luis Potosí,Santa María del Río,Occidente,GUANAJUATILLO,2016-05-01,2016-05-01,1 Día,...,0.00,0.00,19.20,0.00,19.20,11 a 20 Hectáreas,00:04:59,01:09:59,0.201388889,Fumadores
2038,2015,16.714981,-94.746197,Oaxaca,San Miguel Chimalapa,Centro,San Miguel Chimalapa,2015-04-20,2015-04-21,2 a 3 Días,...,0.00,5.00,10.00,15.00,30.00,21 a 50 Hectáreas,00:04:59,00:17:00,0.059027778,Actividades Agrícolas
21571,2018,21.886783,-102.441058,Aguascalientes,Jesús María,Occidente,La Roca,2018-03-02,2018-03-02,1 Día,...,0.00,0.19,1.43,0.00,1.62,0 a 5 Hectáreas,00:04:59,00:04:59,0.100694444,Actividades Ilícitas
56393,2023,20.485833,-98.373889,Veracruz,Zacualpan,Centro,Propiedad Privada De Puerto Chico / La Laja,2023-03-08,2023-03-08,1 Día,...,0.03,0.22,0.00,0.03,0.27,0 a 5 Hectáreas,00:05:00,00:25:00,02:30:00,Actividades Agrícolas
20423,2017,19.196250,-99.272083,Ciudad de México,Tlalpan,Centro,Ajusco,2017-12-20,2017-12-20,1 Día,...,0.00,0.00,0.28,0.00,0.28,0 a 5 Hectáreas,01:05:00,00:54:59,2.364583333,Actividades Pecuarias
53885,2022,20.536667,-98.512222,Veracruz,Huayacocotla,Centro,Ejido La Selva,2022-05-22,2022-05-22,1 Día,...,0.00,0.00,0.42,0.00,0.42,0 a 5 Hectáreas,00:10:00,00:25:00,01:30,Actividades Ilícitas
14158,2017,17.334983,-99.481442,Guerrero,Chilpancingo de los Bravo,Centro,"Zoyatepec, Rincon de La Via, Acahizotla, Part.",2017-03-18,2017-03-25,Mayor de 7 Días,...,192.40,577.20,769.60,192.40,1924.00,Mayor a 100 Hectáreas,00:20:00,00:40:00,0.083333333,Actividades Ilícitas


In [113]:
# Mostramos que ya no existan mas valores nulos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62441 entries, 0 to 62440
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Año               62441 non-null  int64         
 1   Latitud           62441 non-null  float64       
 2   Longitud          62441 non-null  float64       
 3   Estado            62441 non-null  object        
 4   Municipio         62441 non-null  object        
 5   Región            62441 non-null  object        
 6   Predio            62441 non-null  object        
 7   Fecha Inicio      62441 non-null  datetime64[ns]
 8   Fecha Termino     62441 non-null  datetime64[ns]
 9   Duración días     62441 non-null  object        
 10  Tipo de incendio  62441 non-null  object        
 11  Tipo Vegetación   62441 non-null  object        
 12  Régimen de fuego  62441 non-null  object        
 13  Tipo impacto      62441 non-null  object        
 14  Arbolado Adulto   6244

In [83]:
df[df["Duración"]=="eliminar"]

,Año,Latitud,Longitud,Estado,Municipio,Región,Predio,Fecha Inicio,Fecha Termino,Duración días,...,Renuevo,Arbustivo,Herbáceo,Hojarasca,Total hectáreas,Tamaño,Detección,Llegada,Duración,Causa
2,2015,19.273358,-99.056281,Ciudad de México,Xochimilco,Centro,San Gregorio Atlapulco,2015-01-01,2015-01-01,1 Día,...,0.0,1.00,0.00,0.00,1.00,0 a 5 Hectáreas,00:04:59,00:15:00,eliminar,Actividades Ilícitas
4,2015,19.540444,-97.231556,Veracruz,Perote,Centro,Ejido Perote,2015-01-01,2015-01-01,1 Día,...,0.0,0.25,0.00,0.00,0.25,0 a 5 Hectáreas,00:20:00,00:35:00,eliminar,Fumadores
45,2015,20.116750,-105.026611,Jalisco,Tomatlán,Occidente,El guamuchil,2015-01-13,2015-01-14,2 a 3 Días,...,0.0,8.00,0.00,0.00,8.00,6 a 10 Hectáreas,00:04:59,00:35:00,eliminar,Cazadores
68,2015,19.535136,-97.229128,Veracruz,Perote,Centro,Pequeñas Propiedades,2015-01-16,2015-01-16,1 Día,...,0.0,0.50,0.00,0.00,0.50,0 a 5 Hectáreas,04:00:00,10:59:59,eliminar,Fogatas
70,2015,19.009694,-99.838111,Ciudad de México,Coatepec Harinas,Centro,Chiltepec,2015-01-17,2015-01-17,1 Día,...,0.0,0.50,0.00,0.00,0.50,0 a 5 Hectáreas,00:29:59,01:00:00,eliminar,Desconocidas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62424,2023,20.675875,-103.582758,Jalisco,Zapopan,Occidente,Cienega Prieta,2023-12-14,2023-12-14,1 Día,...,0.0,0.00,0.00,0.22,0.22,0 a 5 Hectáreas,01:20:00,eliminar,eliminar,Desconocidas
62427,2023,19.503861,-97.756025,Puebla,Libres,Centro,La Cañada,2023-12-17,2023-12-17,1 Día,...,0.0,0.00,1.74,0.00,1.74,0 a 5 Hectáreas,02:30:00,03:30:00,eliminar,Desconocidas
62433,2023,21.953014,-103.698414,Jalisco,Villa Guerrero,Occidente,Las Guacamayas,2023-12-28,2023-12-28,1 Día,...,0.0,0.80,0.00,2.00,2.80,0 a 5 Hectáreas,03:10:00,04:10:00,eliminar,Intencional
62434,2023,19.778347,-102.464900,Michoacán,Tingüindín,Occidente,C.i. De San Pedro Tinguidin / Amecuaro,2023-12-28,2023-12-28,1 Día,...,0.0,1.20,1.45,0.00,2.65,0 a 5 Hectáreas,00:30:00,00:00:00,eliminar,Actividades Agrícolas


Una vez que comprobado que efectivamente no tenemos más datos nulos y que nuestras respuestas categoricas están normalizadas, ahora podemos continuar con nuestro análisis exploratorio.

## Visualizaciones

Para poder observar las relaciones entre nuestros datos, primero observaremos las distribuciones de nuestras variables por tipo.

In [ ]:
df.iloc[:,9:]

In [ ]:
categorical_variables = df[["Estado", "Municipio", "Región", "Predio","Tipo de incendio", "Tipo Vegetación", "Régimen de fuego"]]
numerical_variables = df[["Latitud", "Longitud"]]
times_variables = df[["Año", "Fecha Inicio", "Fecha Termino","Duración días"]]

In [ ]:
categorical_variables.sample(3)

In [ ]:
# fig, ax = plt.subplots(nrows=6, ncols=3, figsize=(16,12))
# ax = ax.flat
# # Palette
# colors = sns.color_palette(palette = 'tab10', n_colors = len(df.columns)+2)

# for i, column in enumerate(categorical_variables.columns):
sns.countplot(x=categorical_variables["Estado"])
plt.xticks(rotation=90)
#fig.delaxes(ax[7])
# plt.tight_layout()
plt.show()

In [ ]:
categorical_variables.columns

In [ ]:
df.info()

In [ ]:
sns.barplot(df["Fecha Inicio"].value_counts().head(30))
plt.xticks(rotation=70)
plt.ylim(0,150)
plt.show()

How to wirte asfa asfaga 